___ 
### PDF 요약 및 검색 RAG 실습
___

* 참고 문헌 : (https://python.langchain.com/v0.2/docs/tutorials/summarization/)

* RAG 아키텍처

    <img src="images/rag_arch.png" width="1000">

* 문서 요약 방법론 3가지
  
    <img src="https://github.com/ksm26/LangChain-Chat-with-Your-Data/blob/main/images/L5-techniques.png?raw=true" width=600>


In [ ]:
# PDF 관련 라이브러리 설치
# !pip install pypdf unstructured pdf2image pdfminer pypdfium2 pdfminer.six pymupdf pdfplumber amazon-textract-caller

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [149]:
from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import ReduceDocumentsChain, LLMChain, MapReduceDocumentsChain
from langchain.chat_models import ChatOpenAI

from supabase.client import Client, create_client

In [150]:
# 수파베이스 클라이언트 준비
supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

In [151]:
# 임베딩 모델 준비
embeddings = OpenAIEmbeddings()

* 임베딩 벡터 저장테이블 생성 (기존에 생성되어 있으면 생략)

In [59]:
import psycopg2

# 데이터베이스 연결 설정
conn = psycopg2.connect(
    host=os.environ.get("SUPABASE_HOST"),
    port=os.environ.get("SUPABASE_PORT"),
    dbname=os.environ.get("SUPABASE_DB"),
    user=os.environ.get("SUPABASE_USER"),
    password=os.environ.get("SUPABASE_PASSWORD")
)

# 커서 생성
cur = conn.cursor()

# 테이블 생성 쿼리
create_table_query = """
CREATE TABLE IF NOT EXISTS documents (
    id uuid PRIMARY KEY,
    content text not null,
    embedding vector (1536) not null,  -- 1536 크기는 OpenAI에 적합. 필요시 수정하세요.
    metadata jsonb DEFAULT '{}'::jsonb
);
"""

# 테이블 생성 쿼리 실행
cur.execute(create_table_query)

# 검색 함수 쿼리
create_function_query = """
create or replace function search_docs (
  query_embedding vector (1536),
  filter jsonb default '{}'
) returns table (
  id uuid,
  content text,
  metadata jsonb,
  similarity float
) language plpgsql as $$
#variable_conflict use_column
begin
  return query
  select
    id,
    content,
    metadata,
    1 - (documents.embedding <=> query_embedding) as similarity
  from documents
  where metadata @> filter
  order by documents.embedding <=> query_embedding;
end;
$$;
"""

# 검색 함수 쿼리 실행
cur.execute(create_function_query)

# 변경사항 커밋
conn.commit()

# 커서와 연결 종료
cur.close()
conn.close()


  직접 임베딩을 해볼사람은 아래 코드를 실행하면 됩니다.
  ___

In [158]:
# PDF 파일 로드
loader = PyPDFLoader("dataset/docs/abtest.pdf")
document = loader.load()
len(document)


296

In [159]:
# document 임베딩 처리
# embedding_vectors = [embeddings.embed_query(doc.page_content) for doc in document]

In [160]:
# 임베딩 파일 로컬 저장
# import pickle

# # embedding_vectors 로컬에 파일로 저장
# with open('embedding_vectors.pkl', 'wb') as f:
#     pickle.dump(embedding_vectors, f)


In [ ]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=2000, 
    chunk_overlap=500,
)

split_docs = text_splitter.split_documents(document)

len(split_docs)

293

In [ ]:
vector_store = SupabaseVectorStore.from_documents(
    split_docs,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="search_docs",
    chunk_size=200,
)

* 직접하지 않고 이미 임베딩된 데이터를 활용해서 실습해볼 사람은 위 코드를 실행하지 말고 여기부터 진행하세요
____

In [161]:
# 임베딩 데이터 불러오기
import pickle

# 로컬에 저장된 임베딩 데이터 불러오기
with open('embedding_vectors.pkl', 'rb') as f:
    embedding_vectors = pickle.load(f)

# 임베딩 데이터 일부 출력
print(embedding_vectors[:5])



[[0.0015564069914950811, -0.01698177094784678, -0.001229952321404113, 0.002717389980328746, 0.002507416766905854, 0.013867425171908174, -0.001257540077715359, 0.0006640210090707903, 0.02034134236261305, -0.0010360720143358366, 0.005073074262145746, -0.0013701898401210033, -0.0030867588345346894, -0.007687776605754372, -3.1431316752166294e-06, -0.012641304428351161, 0.0146644036086541, -0.012003721865218952, 0.005024029413777012, -0.01817110910286524, 0.013511849904819523, -0.00590683668441422, -0.011562317764239017, -0.003148064778580274, -0.013303409648498401, 0.0009494772257626659, 0.027587716264371478, -0.03962822258077431, 0.02668038633671924, -0.03788712976519226, -0.004720564909260636, 0.007650993085893155, -0.016246098687977106, 0.004463079338909451, -0.005394930992254062, -0.008067874064196725, 0.0008268651630484979, 0.0037274068462091097, 0.010379111437627644, 0.009011987511710184, 0.004395642730610108, -0.02248705389666849, 0.003396354236135489, 0.008208878177879835, -0.01170

In [162]:
# SupabaseVectorStore를 이용하여 documents 테이블에서 데이터 불러오기
vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name="documents",
    query_name="search_docs"
)


___

In [142]:
query = "p-value"
matched_docs = vector_store.similarity_search(query, k=5)

* 맵리듀스 방법론 구현

In [94]:
# LLM 준비
gpt35 = ChatOpenAI(temperature=0,   model_name='gpt-3.5-turbo-16k')

In [111]:
# Map 체인 구성
map_prompt = PromptTemplate(
    template="""Here is a part of the document
    {pages}
    Based on this list of documents, please summarize the main points.
    please in Korean.
    Answer:""",
    input_variables=["pages"]
)

map_chain = LLMChain(llm=gpt35, prompt=map_prompt)

In [133]:
# Reduce 체인 구성
reduce_prompt = PromptTemplate(
    template= """
    Here is a collection of summaries:
    {summaries}
    Based on these, please create a consolidated summary.
    Answer:""",
    input_variables=["summaries"]
)

reduce_chain = LLMChain(llm=gpt35, prompt=reduce_prompt)


In [134]:
# 콜랩스 체인 구성 
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain,                
    document_variable_name="summaries" 
)

In [135]:
# 콜랩스 체인을 연결해서 리듀스 체인 완성
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=4000,
)

In [137]:
# 맵리듀스 체인 완성
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,  # 맵체인
    reduce_documents_chain=reduce_documents_chain, # 리듀스 체인
    document_variable_name="pages",
    return_intermediate_steps=False,
)

In [143]:
# 요약 실행
result = map_reduce_chain.run(matched_docs)


In [144]:
# 결과
from pprint import pprint
pprint(result)

('이 문서는 p-값에 관한 내용을 다루고 있습니다. p-값은 통계적 가설 검정에서 사용되는 지표로, 관찰된 결과보다 극단적인 결과가 나올 '
 '확률을 나타냅니다. 그러나 p-값을 잘못 해석하면 오해가 발생할 수 있으며, 신뢰구간에 포함된 값들은 실제 값이 될 확률을 나타내는 것이 '
 '아니라는 점을 주의해야 합니다. 이러한 지표들은 데이터 기반 비즈니스 의사결정에 도움을 줄 수 있습니다.')
